In [45]:
import torch
import torchtext
from torchtext.data import get_tokenizer
import pandas as pd

DEVICE = torch.device("cuda") if torch.cuda.is_available() else "cpu"

# load the dataset
path_train = "datasets/train.csv"
df_train = pd.read_csv(filepath_or_buffer=path_train)
tweets = df_train["text"]
first_tweet = tweets[0] # for experimentation, can be deleted later TODO

# create the dictionary of tokens in the tweets
tokenizer = get_tokenizer("basic_english")
tokens = [] 
for tweet in tweets:
	tokens += tokenizer(tweet)
wtoi, itow = {}, {}
for i, w in enumerate(tokens):
	wtoi[w] = i+1
	itow[i+1] = w
wtoi["UNKNOWN"] = 0 # to deal with words not seen in train set
itow[0] = "UNKNOWN"
# helper function to tansform each token in a tweet to a unique integer
def tweetEncoder(tweet):
	tokens = tokenizer(tweet)
	for i in range(len(tokens)):
		tokens[i] = wtoi[tokens[i]] if tokens[i] in wtoi else 0
	return tokens

print(tweetEncoder(first_tweet)) # test for the first tweet

# create a pandas serie with these encoding for each tweet
x_train = tweets.apply(lambda tweet: tweetEncoder(tweet))
y_train = df_train["target"]

len(x_train) == len(y_train)


[143624, 95295, 144116, 144655, 137092, 144626, 143598, 144385, 144425, 122629, 42489, 142287, 144025]


True

In [43]:
df_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
